In [1]:
import pandas as pd
from astral.sun import sun
from astral import LocationInfo
from datetime import date, datetime, timedelta
from pysolar.solar import get_altitude

In [2]:
df = pd.read_csv('data.csv')
df.sort_values(by=['date'], inplace=True)
df.head()

,date,water_temp,cloudiness_0,precipitation_0,temp_0,wind_dir_0,wind_speed_0,wind_dir_x_0,wind_dir_y_0,pressure_0,...,wind_dir_y_21,pressure_21,cloudiness_24,precipitation_24,temp_24,wind_dir_24,wind_speed_24,wind_dir_x_24,wind_dir_y_24,pressure_24
0,2011-01-01,0.3,пасмурно,без осадков,-1.0,З,3.0,-5.510911e-16,-3.000000e+00,754.0,...,1.414214e+00,755.0,малооблачно,слабые осадки,-3.0,ЮВ,2.0,-1.414214,1.414214e+00,754.0
1,2011-01-02,0.3,малооблачно,слабые осадки,-3.0,ЮВ,2.0,-1.414214e+00,1.414214e+00,754.0,...,3.826834e-01,754.0,пасмурно,без осадков,-4.0,ЮВ,1.0,-0.707107,7.071068e-01,754.0
2,2011-01-03,0.2,пасмурно,без осадков,-4.0,ЮВ,1.0,-7.071068e-01,7.071068e-01,754.0,...,2.449294e-16,753.0,ясно,без осадков,-5.0,ЮЮЗ,2.0,-1.847759,-7.653669e-01,753.0
3,2011-01-04,2.3,ясно,без осадков,-5.0,ЮЮЗ,2.0,-1.847759e+00,-7.653669e-01,753.0,...,7.653669e-01,752.0,пасмурно,без осадков,-5.0,Ю,2.0,-2.000000,2.449294e-16,753.0
4,2011-01-05,2.1,пасмурно,без осадков,-5.0,Ю,2.0,-2.000000e+00,2.449294e-16,753.0,...,-7.071068e-01,754.0,ясно,без осадков,-5.0,ЮЗ,1.0,-0.707107,-7.071068e-01,755.0


In [3]:
df['month'] = df['date'].apply(lambda x: x.split('-')[1])
df['day'] = df['date'].apply(lambda x: x.split('-')[2])
for col in df.columns:
    if df[col].dtype != 'O':
        df[col] = df.groupby(['day', 'month'])[col].transform(lambda x: x.fillna(x.mean()))
        df[col] = df.groupby('month')[col].transform(lambda x: x.fillna(x.mean()))

In [4]:
df.dropna(inplace=True, axis=0)

In [5]:
latitude, longitude = 59.5, 29.1
city = LocationInfo("Sosnovy Bor", "Russia", "Europe/Moscow", latitude, longitude)

def get_day_length(x):

    s = sun(city.observer, date.fromisoformat(x))
    return (s['sunset'] - s['sunrise']).seconds / 3600



def solar_elevation(time):

    s = sun(city.observer, date.fromisoformat(time))    
    return get_altitude(latitude, longitude, s['noon'])



df['day_length'] = df['date'].apply(get_day_length)

df['solar_elevation'] = df['date'].apply(solar_elevation)

In [6]:
df.head()

,date,water_temp,cloudiness_0,precipitation_0,temp_0,wind_dir_0,wind_speed_0,wind_dir_x_0,wind_dir_y_0,pressure_0,...,temp_24,wind_dir_24,wind_speed_24,wind_dir_x_24,wind_dir_y_24,pressure_24,month,day,day_length,solar_elevation
0,2011-01-01,0.3,пасмурно,без осадков,-1.0,З,3.0,-5.510911e-16,-3.000000e+00,754.0,...,-3.0,ЮВ,2.0,-1.414214,1.414214e+00,754.0,01,01,6.168333,7.600029
1,2011-01-02,0.3,малооблачно,слабые осадки,-3.0,ЮВ,2.0,-1.414214e+00,1.414214e+00,754.0,...,-4.0,ЮВ,1.0,-0.707107,7.071068e-01,754.0,01,02,6.199444,7.684113
2,2011-01-03,0.2,пасмурно,без осадков,-4.0,ЮВ,1.0,-7.071068e-01,7.071068e-01,754.0,...,-5.0,ЮЮЗ,2.0,-1.847759,-7.653669e-01,753.0,01,03,6.233333,7.775738
3,2011-01-04,2.3,ясно,без осадков,-5.0,ЮЮЗ,2.0,-1.847759e+00,-7.653669e-01,753.0,...,-5.0,Ю,2.0,-2.000000,2.449294e-16,753.0,01,04,6.269722,7.874868
4,2011-01-05,2.1,пасмурно,без осадков,-5.0,Ю,2.0,-2.000000e+00,2.449294e-16,753.0,...,-5.0,ЮЗ,1.0,-0.707107,-7.071068e-01,755.0,01,05,6.308333,7.981454


In [8]:
print(df.shape)

(4380, 78)


In [7]:
df.to_csv('data_prepared.csv', index=False)